# **Convolutional Neural Network**
**Starting parameters**:

- Number of epochs: 100
- Learning rate: 0.0005
- Layers: [64, 32, 32]
- Dropout: 0.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For preprocessing
import tensorflow as tf

# For modeling
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import random

In [ ]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_3s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)
del file

In [ ]:
train_data = data['train'].copy()
val_data = data['val'].copy()
del data

train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

## **Shuffling Data**

In [ ]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [ ]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [ ]:
val_y, val_features = shuffle_data(val_labels, v_features)

## **FFNN Model**

In [22]:
def build_model(audio_features,
                hidden_layers=[256, 128, 64],
                learning_rate=0.0001,
                dropout_rate=0.4,
                regularizer_val=0.002,
                output_units=20
                ):

  tf.keras.backend.clear_session()

  # set input layer
  input = tf.keras.Input(shape=(audio_features.shape[1],), name='Input')

  # set hidden layers
  hidden_output = input

  for i, hidden_dim in enumerate(hidden_layers):
    hidden_output = tf.keras.layers.Dense(units=hidden_dim,
                                          activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l2(regularizer_val),
                                          name=f'hidden{i}')(hidden_output)
    hidden_output = tf.keras.layers.Dropout(rate=dropout_rate, name=f'dropout{i}')(hidden_output)

  # set output layer
  output = tf.keras.layers.Dense(units=output_units,
                                 activation='softmax',
                                 name='Output')(hidden_output)
  # create model
  model = tf.keras.models.Model(inputs=input, outputs=[output])

  # compile model
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                metrics=['accuracy'])

  return model

In [23]:
def tile_and_crop(feature, target_size):
    tiled = np.tile(feature, (1, target_size // feature.shape[1] + 1, 1))
    return tiled[:, :target_size, :]

In [24]:
train_results = {}
val_results = {}

val_scores = {}

### With Mel Spectrogram, MFCCs and Chroma

In [25]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 41440)

In [26]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 41440)

In [27]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)

model = build_model(training_features)

history = model.fit(
    x=training_features,
    y=train_y,
    epochs=60,
    validation_data=(
        validation_features,
        val_y,
    ),
    batch_size=32,
    verbose=1
)

Epoch 1/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 10s 39ms/step - accuracy: 0.1237 - loss: 6.6585 - val_accuracy: 0.5230 - val_loss: 3.2199
Epoch 2/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.2808 - loss: 4.8653 - val_accuracy: 0.6022 - val_loss: 2.9744
Epoch 3/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3583 - loss: 4.3166 - val_accuracy: 0.5778 - val_loss: 2.9515
Epoch 4/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3948 - loss: 4.0375 - val_accuracy: 0.6089 - val_loss: 2.8705
Epoch 5/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4356 - loss: 3.7028 - val_accuracy: 0.5985 - val_loss: 2.8860
Epoch 6/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4731 - loss: 3.5795 - val_accuracy: 0.6000 - val_loss: 2.8146
Epoch 7/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4676 - loss: 3.4629 - val_accuracy: 0.6281 - val_loss: 2.8245
Epoch 8/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5028 - loss: 3.3539 - val_accuracy: 

In [28]:
model.save('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/FFNN/FFNN_melspectrogram_chroma_mfcc.keras')

In [29]:
train_yhat = model.predict(training_features)
train_yhat_result = np.argmax(train_yhat, axis=-1)
train_results['melspectrogram_chroma_mfcc'] = model.evaluate(training_features, train_y)[-1]

val_yhat = model.predict(validation_features)
val_yhat_result = np.argmax(val_yhat, axis=-1)
val_results['melspectrogram_chroma_mfcc'] = model.evaluate(validation_features, val_y)[-1]

165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9861 - loss: 0.8843
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7454 - loss: 1.8895


### With Mel Spectrogram, MFCCs and CQT

In [30]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['cqt'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 60088)

In [31]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['cqt'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 60088)

In [32]:
scaler = StandardScaler()
training_features = scaler.fit_transform(training_features)
validation_features = scaler.transform(validation_features)

model = build_model(training_features)

history = model.fit(
    x=training_features,
    y=train_y,
    epochs=60,
    validation_data=(
        validation_features,
        val_y,
    ),
    batch_size=32,
    verbose=1
)

Epoch 1/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.1267 - loss: 8.0371 - val_accuracy: 0.4852 - val_loss: 3.3864
Epoch 2/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2360 - loss: 6.1245 - val_accuracy: 0.4689 - val_loss: 3.2534
Epoch 3/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.2865 - loss: 5.2992 - val_accuracy: 0.4867 - val_loss: 3.1967
Epoch 4/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3274 - loss: 4.8576 - val_accuracy: 0.4830 - val_loss: 3.2513
Epoch 5/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3447 - loss: 4.5747 - val_accuracy: 0.5696 - val_loss: 3.0928
Epoch 6/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3956 - loss: 4.1151 - val_accuracy: 0.5748 - val_loss: 3.0068
Epoch 7/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.3935 - loss: 4.0508 - val_accuracy: 0.5926 - val_loss: 2.9584
Epoch 8/60
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4245 - loss: 3.9344 - val_accuracy: 

In [33]:
model.save('/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/ML Models/FFNN/FFNN_melspectrogram_cqt_mfcc.keras')

In [34]:
train_yhat = model.predict(training_features)
train_yhat_result = np.argmax(train_yhat, axis=-1)
train_results['melspectrogram_cqt_mfcc'] = model.evaluate(training_features, train_y)[-1]

val_yhat = model.predict(validation_features)
val_yhat_result = np.argmax(val_yhat, axis=-1)
val_results['melspectrogram_cqt_mfcc'] = model.evaluate(validation_features, val_y)[-1]

165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9643 - loss: 1.0359
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7232 - loss: 1.9679


## Review all the results

In [35]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,melspectrogram_chroma_mfcc,0.987,0.737
1,melspectrogram_cqt_mfcc,0.965,0.719
